# Record entire power runs
Goal is to screen cap and also merge the webcam feed looking at power display

In [91]:
import cv2
import cupy as np
import csv
import pytesseract
import easyocr

reader = easyocr.Reader(
    ["en"]
)  # this needs to run only once to load the model into memory
# def clean_square_for_OCR_2(image):
#     grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     thresh = cv2.threshold(grey, 0, 200, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
#     kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
#     opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
#     cnts = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     cnts = cnts[0] if len(cnts) == 2 else cnts[1]
#     for c in cnts:
#         area = cv2.contourArea(c)
#         if area < 50:
#             cv2.drawContours(opening, [c], -1, 0, -1)

#     result = 240 - opening
#     result = opening
#     result = cv2.GaussianBlur(result, (5, 5), 0)
#     return result


def clean_square_for_OCR(image):
    grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(grey, 0, 100, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
    cnts = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        area = cv2.contourArea(c)
        if area < 50:
            cv2.drawContours(opening, [c], -1, 0, -1)

    result = 240 - opening
    result = opening
    result = cv2.GaussianBlur(result, (5, 5), 0)
    return result


# def clean_square_for_OCR(image):
#     # Convert to grayscale
#     grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
#     # Convert to HSV and isolate blue channel
#     hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
#     blue_channel = hsv[:, :, 0]  # Extract the hue channel
    
#     # Threshold the blue channel to isolate blue regions
#     _, blue_mask = cv2.threshold(blue_channel, 100, 140, cv2.THRESH_BINARY)
    
#     # Apply the mask to the grayscale image
#     filtered = cv2.bitwise_and(grey, grey, mask=blue_mask)
    
#     # Remove noise
#     denoised = cv2.medianBlur(filtered, 5)

#     # Threshold the denoised image
#     _, thresh = cv2.threshold(filtered, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
#     kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
#     opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
#     cnts = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     cnts = cnts[0] if len(cnts) == 2 else cnts[1]
#     for c in cnts:
#         area = cv2.contourArea(c)
#         if area < 50:
#             cv2.drawContours(opening, [c], -1, 0, -1)
    
#     # Invert the image
#     result = 240 - opening
#     result = opening
#     result = cv2.GaussianBlur(result, (5, 5), 0)
#     return result


mypath = "../files/"
run = mypath + "runs.csv"
run = "../tmp/runs.csv"
# name = "test"
name = "test"
inname = mypath + f"{name}.mp4"
otname = mypath + f"{name}_clean.mp4"
exp_expected = 4

cap = cv2.VideoCapture(inname)
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
fps = 20.0
size = (1280, 960)
out = cv2.VideoWriter(otname, fourcc, fps, size)

runfile = open(run, "r")
runreader = csv.reader(runfile)
runlist = list(runreader)
csvfile = open(mypath + f"{name}_clean.csv", "w")
csvwriter = csv.writer(csvfile)
csvwriter.writerow(
    [
        "model",
        "hw",
        "version",
        "delegate",
        "del_version",
        "thread",
        "num_runs",
        "start_power",
        "end_power",
    ]
)


exp_count = 0
curr_in_start = False
curr_in_power = ""
while True:
    if exp_count == exp_expected:
        break
    enter_frame = False
    exit_frame = False

    ret, frame = cap.read()
    if not ret:
        break

    # check = frame[885:910, 20:230, :]
    check = frame[880:920, 20:230, :]

    check = cv2.resize(check, (check.shape[1] * 4, check.shape[0] * 4))
    ncheck = check

    # Check for start and end of run
    ocr = reader.readtext(ncheck)
    ocr = " ".join([x[1] for x in ocr])
    ocr = ocr.lower()
    check = cv2.putText(
        img=np.copy(ncheck),
        text=ocr,
        org=(0, 10),
        fontFace=2,
        fontScale=0.5,
        color=(0, 255, 0),
        thickness=1,
    )

    keyword_check = True if "start" in ocr and "power" in ocr else False
    if keyword_check and not curr_in_start:
        curr_in_start = True
        enter_frame = True
        print("Started")

    elif not keyword_check and curr_in_start:
        curr_in_start = False
        exit_frame = True
        exp_count += 1
        print("Ended")

    ## OCR for power
    power = frame[35:110, 900:-150, :]  # y , x // adjust to focus on mWH
    power = cv2.resize(power, (power.shape[1] * 4, power.shape[0] * 4))
    npower = clean_square_for_OCR(power)
    power_ocr = pytesseract.image_to_string(
        npower, lang="eng", config="--psm 8 -c tessedit_char_whitelist=0123456789"
    ).replace("\n\x0c", "")
    print(power_ocr)
    cv2.imshow("npower", npower)
    if cv2.waitKey(1) == ord("q"):
        break

    # Don't save run frames except for the start and end
    if not enter_frame and not exit_frame:
        continue

    power_ocr = pytesseract.image_to_string(
        npower, lang="eng", config="--psm 8 -c tessedit_char_whitelist=0123456789"
    ).replace("\n\x0c", "")
    power_ocr = int(power_ocr) if power_ocr else 0

    if enter_frame:
        curr_in_power = power_ocr
    if exit_frame:
        row = runlist[exp_count - 1] + [curr_in_power, power_ocr]
        csvwriter.writerow(row)
    out.write(frame)


# Release resources and close the video writer
cv2.destroyAllWindows()
cap.release()
out.release()
csvfile.close()



11994291
1124291
1124291
1124291
1129450
1129450
119942
1129432
1129452
1129452
1199453
1199453
11294541
11994541
1139455
1139455
11294561
11294561
11294561
1128450
1128450
1128458
1199458
11294591
11994591
1129440
1199440
11984411
1198441
11994411
1199441
1199445
119944
1199445
112944
1199444
1199444
11294461
11294461
11294461
119944
112844
119944
1199448
1129448
11294491
119844
112450
1199456
11934511
119845
119452
11994521
112453
119845041
11294541
11294541
11294541
1129454
1124551
112453
11284511
1128451
1129451
11294521
